In [1]:
path = 'data/emails.txt'
with open(path, 'rb') as f:
    text = f.read().decode('utf-8', errors='ignore')

In [2]:
import re

def remove_html_tags(text_with_html):
    # Define a regular expression pattern to match HTML tags
    html_tag_pattern = re.compile('<.*?>')

    # Use the `sub` method to replace all matched HTML tags with an empty string
    text_without_html = re.sub(html_tag_pattern, '', text_with_html)

    return text_without_html

text = remove_html_tags(text)

In [3]:
import regex

def cleanup(dirty_text):

    # Define a regular expression pattern to match nested square brackets
    square_brackets_pattern = regex.compile(r'\[(?:[^[\]]*|(?R))*\]')
    dirty_text = regex.sub(square_brackets_pattern, '', dirty_text)

    # Define a regular expression pattern to match text inside square brackets
    square_brackets_pattern = re.compile(r'\[.*?\]')
    dirty_text = re.sub(square_brackets_pattern, '', dirty_text)

    # Weird character
    dirty_text = dirty_text.replace("=E2=80=99", "`")

    # Define a regular expression pattern to match '=E2=80=' followed by a number
    e2_80_sequence_pattern = re.compile(r'=E2=80=\d+')
    dirty_text = re.sub(e2_80_sequence_pattern, '', dirty_text)

    # Define a regular expression pattern to match sequences starting with '=' and ending with a number
    equals_sequence_pattern = re.compile(r'=([A-F0-9]{2}=)+\d+')
    dirty_text = re.sub(equals_sequence_pattern, '', dirty_text)

    # Define a regular expression pattern to match '=' followed by any non-letter characters
    equals_non_letters_pattern = re.compile('=([^a-zA-Z]+)')
    dirty_text = re.sub(equals_non_letters_pattern, '', dirty_text)

    clean_text = dirty_text.replace("=E2", "`")

    return clean_text

def remove_minute_read(text):
    # Define a regular expression pattern to match the sequence
    minute_read_pattern = re.compile(r'\(?\d*\s*MINUTE\s*READ\)?', re.IGNORECASE)

    # Use the `sub` method to replace all matched sequences with an empty string
    clean_text = re.sub(minute_read_pattern, '', text)

    return clean_text

text = cleanup(text)
text = remove_minute_read(text)

In [4]:
arguments = [
    "BIG TECH & STARTUPS",
    "SCIENCE & FUTURISTIC TECHNOLOGY",
    "PROGRAMMING, DESIGN & DATA SCIENCE",
    "MISCELLANEOUS"
]

def extract_sections(text, start_section, end_section):
    # Define the regular expression pattern to match the desired sections
    pattern = re.compile(r'(?i){}(.*?)(?={})'.format(re.escape(start_section), re.escape(end_section)), re.MULTILINE | re.DOTALL)
    sections = re.findall(pattern, text)

    return sections

In [5]:
text_by_argument = {}

# Iterate through the sections_to_extract list using indices
for i in range(len(arguments) - 1):
    argument = arguments[i]
    next_argument = arguments[i + 1]

    text_by_argument[argument] = extract_sections(text, argument, next_argument)

for section, content in text_by_argument.items():
    print(f"{section}: {len(content)} sections extracted")

BIG TECH & STARTUPS: 213 sections extracted
SCIENCE & FUTURISTIC TECHNOLOGY: 212 sections extracted
PROGRAMMING, DESIGN & DATA SCIENCE: 240 sections extracted


In [6]:
#pattern  to separate 2 articles that are paired together
pattern = r"\.(?=\b[A-Z]{2,})"

#foreach article found add it to a list of articles
articles_by_argument = {}

for i in range(len(arguments) - 1):
    argument = arguments[i]
    articles_by_argument[argument] = []
    for j, section in enumerate(text_by_argument[argument], start=1):
        articles = re.split(pattern, section)
        for art in articles:
            articles_by_argument[argument].append(art)


In [13]:
dataSplitInto = [
    "training",
    "test",
    "validation"
]

def trainingTestValidation(trainPercent, testPercent):
    text_dict = {}
    
    for splitArea in dataSplitInto:
        text_dict[splitArea] = []
    
    for i in range(len(arguments) - 1):
        argument = arguments[i]
        value = articles_by_argument[argument]
        
        p_train = int(len(value) * trainPercent)
        p_test = int(len(value) * testPercent)
        p_valid = len(value) - p_train - p_test
        
        for i, section in enumerate(value[:p_train], start=1):
            text_dict["training"].append(section)
        
        for i, section in enumerate(value[p_train:p_train+p_test], start=1):
            text_dict["test"].append(section)
        
        for i, section in enumerate(value[-p_valid:], start=1):
            text_dict["validation"].append(section)      
                    
    return text_dict

separated_text = trainingTestValidation(0.1,0.1)

for key, value in separated_text.items():
    print(key, " has ", len(value))

training  has  197
test  has  197
validation  has  1586
